# Exploring Chat Templates with SmolLM2

This notebook demonstrates how to use chat templates with the `SmolLM2` model. Chat templates help structure interactions between users and AI models, ensuring consistent and contextually appropriate responses.

## 환경 설치
- uv가 이미 설치되어 있어 좀 더 빠르게 설치 가능

In [ ]:
!uv pip install transformers datasets trl huggingface_hub --system

In [5]:
import os
from getpass import getpass
os.environ['HF_TOKEN'] = getpass()

··········


In [ ]:
""

In [ ]:
os.environ['HF_TOKEN']

In [7]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format
import torch

## SmolLM2 Chat Template

Let's explore how to use a chat template with the `SmolLM2` model. We'll define a simple conversation and apply the chat template.

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# checkpoint = "HuggingFaceTB/SmolLM2-135M"
checkpoint = "Qwen/Qwen2.5-0.5B"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, token=os.environ['HF_TOKEN'] )
model = AutoModelForCausalLM.from_pretrained(checkpoint, token=os.environ['HF_TOKEN']).to(device)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [9]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-0.5B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, norma

In [10]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

- [setup_chat_format](https://github.com/huggingface/trl/blob/6e088d165c5f3e7d8b20bd7b7248b8dcf32dbf5d/trl/models/utils.py#L79-L135) 이 하는 건 Embedding Token 교체하는 것

```py
# check if model already had a chat template
if tokenizer.chat_template is not None:
    raise ValueError(
        "Chat template is already added to the tokenizer. If you want to overwrite it, please set it to None"
    )
```

- chat_templates가 이미 존재하는 경우에는 해당 templates로 따름


In [11]:
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

ValueError: Chat template is already added to the tokenizer. If you want to overwrite it, please set it to None

In [12]:
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
]

# Apply chat template without tokenization

The tokenizer represents the conversation as a string with special tokens to describe the role of the user and the assistant.


In [13]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)

print("Conversation with template:", input_text)

Conversation with template: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>



In [14]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)

print("Conversation with template:", input_text)

Conversation with template: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>



# Decode the conversation

Note that the conversation is represented as above but with a further assistant message.


In [25]:
input_text = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True
)

print("Conversation decoded:", tokenizer.decode(token_ids=input_text))

Conversation decoded: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>
<|im_start|>assistant



In [26]:
input_text

[1,
 4093,
 198,
 19556,
 28,
 638,
 359,
 346,
 47,
 2,
 198,
 1,
 520,
 9531,
 198,
 57,
 5248,
 2567,
 876,
 28,
 9984,
 346,
 17,
 1073,
 416,
 339,
 4237,
 346,
 1834,
 47,
 2,
 198,
 1,
 520,
 9531,
 198]

# Tokenize the conversation

Of course, the tokenizer also tokenizes the conversation and special token as ids that relate to the model's vocabulary.



In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Conversation tokenized:", input_text)

Conversation tokenized: [1, 4093, 198, 19556, 28, 638, 359, 346, 47, 2, 198, 1, 520, 9531, 198, 57, 5248, 2567, 876, 28, 9984, 346, 17, 1073, 416, 339, 4237, 346, 1834, 47, 2, 198, 1, 520, 9531, 198]


In [43]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Conversation tokenized:", input_text)

Conversation tokenized: [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 9707, 11, 1246, 525, 498, 30, 151645, 198, 151644, 77091, 198, 40, 2776, 3730, 1632, 11, 9702, 498, 0, 2585, 646, 358, 7789, 498, 3351, 30, 151645, 198, 151644, 77091, 198]


In [29]:
tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)

[1,
 4093,
 198,
 19556,
 28,
 638,
 359,
 346,
 47,
 2,
 198,
 1,
 520,
 9531,
 198,
 57,
 5248,
 2567,
 876,
 28,
 9984,
 346,
 17,
 1073,
 416,
 339,
 4237,
 346,
 1834,
 47,
 2,
 198,
 1,
 520,
 9531,
 198]

In [32]:
tokenizer.decode(token_ids=tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True))

"<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing well, thank you! How can I assist you today?<|im_end|>\n<|im_start|>assistant\n"

In [31]:
tokenizer.decode(token_ids=tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=False))

"<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing well, thank you! How can I assist you today?<|im_end|>\n"

In [34]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

"<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing well, thank you! How can I assist you today?<|im_end|>\n"

In [35]:
tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=True)

"<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing well, thank you! How can I assist you today?"

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Process a dataset for SFT</h2>
    <p>Take a dataset from the Hugging Face hub and process it for SFT. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Convert the `HuggingFaceTB/smoltalk` dataset into chatml format.</p>
    <p>🐕 Convert the `openai/gsm8k` dataset into chatml format.</p>
</div>

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/HuggingFaceTB/smoltalk/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [47]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations", token=os.environ["HF_TOKEN"])


def process_dataset(sample):
  chat_template = tokenizer.apply_chat_template(sample["messages"], tokenize=False)
  return {"chatml": chat_template}

ds = ds.map(process_dataset)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [49]:
ds

In [ ]:
display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/openai/gsm8k/embed/viewer/main/train"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [58]:
ds = load_dataset("openai/gsm8k", "main", token=os.environ["HF_TOKEN"])


def process_dataset(sample):
    messages = [
        {"role": "user", "content": sample["question"]},
        {
            "role": "assistant",
            "content": sample["answer"],
        },
    ]
    chat_template = tokenizer.apply_chat_template(messages, tokenize=False)
    return  {"messages": messages}


ds = ds.map(process_dataset)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [59]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'messages'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer', 'messages'],
        num_rows: 1319
    })
})

## Conclusion

This notebook demonstrated how to apply chat templates to different models, `SmolLM2`. By structuring interactions with chat templates, we can ensure that AI models provide consistent and contextually relevant responses.

In the exercise you tried out converting a dataset into chatml format. Luckily, TRL will do this for you, but it's useful to understand what's going on under the hood.

# Supervised Fine-Tuning with SFTTrainer

- SFTTrainer를 활용하여 LLM을 SFT하고 실제 Base Model에서 대답을 할 수 있도록 튜닝

In [17]:
finetune_name = "Qweb2.5-FT-CSY"
finetune_tags = ["smol-course", "module_1"]

In [15]:
prompt = "프로그래밍에 대한 시를 써줘"
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)

print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Before training:
system
You are a helpful assistant.
user
프로그래밍에 대한 시를 써줘
ocoder
ocoder
ocoder
ocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoderocoder


In [25]:
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset

In [23]:
ds = load_dataset("beomi/KoAlpaca-RealQA", "default", token=os.environ["HF_TOKEN"])


def process_dataset(sample):
    messages = [
        {"role": "user", "content": sample["question"]},
        {
            "role": "assistant",
            "content": sample["answer"],
        },
    ]
    # Qwen은 알아서 처리함
    return  {"messages": messages}


ds = ds.map(process_dataset)

train-00000-of-00001.parquet:   0%|          | 0.00/14.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18524 [00:00<?, ? examples/s]

Map:   0%|          | 0/18524 [00:00<?, ? examples/s]

In [26]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=500,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
)

<ipython-input-26-7a5c1af5e8be>:13: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/18524 [00:00<?, ? examples/s]

In [27]:
trainer.train()
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: csy1204 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,2.351200
20,2.231700
30,2.084100
40,1.975200
50,2.094400
60,2.014200
70,1.836000
80,2.020000
90,1.996300
100,1.916500


In [63]:
trainer.push_to_hub(tags=finetune_tags, token=os.environ["HF_TOKEN"])

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

events.out.tfevents.1738433012.25a145720b49.4555.0:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/josang1204/Qweb2.5-FT-CSY/commit/8543876d9b277cb0c7b689df0607fc93a58f7f56', commit_message='End of training', commit_description='', oid='8543876d9b277cb0c7b689df0607fc93a58f7f56', pr_url=None, repo_url=RepoUrl('https://huggingface.co/josang1204/Qweb2.5-FT-CSY', endpoint='https://huggingface.co', repo_type='model', repo_id='josang1204/Qweb2.5-FT-CSY'), pr_revision=None, pr_num=None)

## Liger Kernel

- 더 많은 Throughput을 내는 Liger kernel로 테스트
- Single GPU라 그런지 딱히 성능 향상은 없음

In [34]:
# pip install liger-kernel

In [68]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output_liger",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    hub_model_id=f"{finetune_name}_liger",  # Set a unique name for your model
    use_liger=True
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-68-0e277e84fcd2>:16: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [69]:
trainer.train()
trainer.save_model(f"./{finetune_name}_liger")

Step,Training Loss,Validation Loss
50,0.580800,1.125099
100,0.597400,1.147599
150,0.587300,1.146946
200,0.576700,1.153045
250,0.600800,1.131423
300,0.593500,1.124247
350,0.614800,1.128012
400,0.716200,1.079427
450,0.787500,1.038132
500,0.908900,0.993534


## Test

In [29]:
# TODO: base 모델과의 차이를 비교해보세요.

prompt = "프로그래밍에 대한 시를 써줘"
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [31]:
outputs = model.generate(**inputs, max_new_tokens=1000)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


After training:
system
You are a helpful assistant.
user
프로그래밍에 대한 시를 써줘
assistant
프로그래밍은 컴퓨터 과학의 한 분야로, 컴퓨터가 어떻게 문제를 해결하고 데이터를 처리하는지를 설명하는 데 중점을 둡니다. 프로그래밍은 다양한 언어와 도구를 통해 구현되며, 이를 통해 다양한 문제를 해결하고 데이터를 처리할 수 있습니다. 다음은 프로그래밍에 대한 시를 작성한 몇 가지 주제입니다.

1. **프로그래밍의 개념과 기초**: 프로그래밍은 어떻게 구현되나요? 어떤 언어와 도구를 사용하나요? 프로그래밍의 기본 개념을 설명해 주세요.

2. **프로그래밍의 주요 개념**: 프로그래밍은 어떻게 문제를 해결하고 데이터를 처리하는지를 설명합니다. 예를 들어, 어떤 문제를 해결하는지, 어떤 데이터를 처리하는지, 그리고 어떻게 데이터를 처리하는지를 설명해 주세요.

3. **프로그래밍의 도구와 기술**: 프로그래밍은 어떻게 도구와 기술을 사용하여 문제를 해결하고 데이터를 처리하는지를 설명합니다. 예를 들어, 어떤 도구를 사용하는지, 어떤 기술을 사용하는지, 그리고 그 기능을 설명해 주세요.

4. **프로그래밍의 역사와 발전**: 프로그래밍은 어떻게 발전하고 어떻게 기술적 발전을 이끌어왔나요? 프로그래밍의 역사와 발전을 설명해 주세요.

5. **프로그래밍의 장점과 단점**: 프로그래밍은 어떤 장점과 단점이 있으며, 어떤 환경에서 더 효율적일까요? 프로그래밍의 장점과 단점에 대해 설명해 주세요.

6. **프로그래밍의 활용**: 프로그래밍은 어떻게 활용되나요? 예를 들어, 어떤 프로그래밍 언어나 도구가 사용되는지, 그리고 그 활용 방식을 설명해 주세요.

7. **프로그래밍의 교육과 인식**: 프로그래밍은 어떻게 교육과 인식을 제공하는지를 설명합니다. 예를 들어, 어떤 교육 프로그램이나 인식 프로그램이 사용되는지, 그리고 그 목적과 효과를 설명해 주세요.

8. **프로그래밍의 사회적, 경제적 영향**: 프로그래밍은 어

In [33]:
tokenizer.decode(outputs[0])

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n프로그래밍에 대한 시를 써줘<|im_end|>\n<|im_start|>assistant\n프로그래밍은 컴퓨터 과학의 한 분야로, 컴퓨터가 어떻게 문제를 해결하고 데이터를 처리하는지를 설명하는 데 중점을 둡니다. 프로그래밍은 다양한 언어와 도구를 통해 구현되며, 이를 통해 다양한 문제를 해결하고 데이터를 처리할 수 있습니다. 다음은 프로그래밍에 대한 시를 작성한 몇 가지 주제입니다.\n\n1. **프로그래밍의 개념과 기초**: 프로그래밍은 어떻게 구현되나요? 어떤 언어와 도구를 사용하나요? 프로그래밍의 기본 개념을 설명해 주세요.\n\n2. **프로그래밍의 주요 개념**: 프로그래밍은 어떻게 문제를 해결하고 데이터를 처리하는지를 설명합니다. 예를 들어, 어떤 문제를 해결하는지, 어떤 데이터를 처리하는지, 그리고 어떻게 데이터를 처리하는지를 설명해 주세요.\n\n3. **프로그래밍의 도구와 기술**: 프로그래밍은 어떻게 도구와 기술을 사용하여 문제를 해결하고 데이터를 처리하는지를 설명합니다. 예를 들어, 어떤 도구를 사용하는지, 어떤 기술을 사용하는지, 그리고 그 기능을 설명해 주세요.\n\n4. **프로그래밍의 역사와 발전**: 프로그래밍은 어떻게 발전하고 어떻게 기술적 발전을 이끌어왔나요? 프로그래밍의 역사와 발전을 설명해 주세요.\n\n5. **프로그래밍의 장점과 단점**: 프로그래밍은 어떤 장점과 단점이 있으며, 어떤 환경에서 더 효율적일까요? 프로그래밍의 장점과 단점에 대해 설명해 주세요.\n\n6. **프로그래밍의 활용**: 프로그래밍은 어떻게 활용되나요? 예를 들어, 어떤 프로그래밍 언어나 도구가 사용되는지, 그리고 그 활용 방식을 설명해 주세요.\n\n7. **프로그래밍의 교육과 인식**: 프로그래밍은 어떻게 교육과 인식을 제공하는지를 설명합니다. 예를 들어, 어떤 교육 프로그램이나 인식 프로그램이 사용되는지,